<center><h1 style="color:#173F8A;"> Metodos para Ecuaciones Diferenciales, IMT3410, 2022-2 </h1></center>
<h3 style="color:#173F8A;text-align:right;"> Instituto de Ingenieria Matematica y Computacional<br>  Pontificia Universidad Catolica de Chile <br>  </h3>
<h3 style="color:#0176DE;text-align:right;"> Profesor. Manuel A. Sanchez<br> </h3>
<hr style="border:2px solid #03122E"> </hr>

<!-- Palette colors UC: celeste:#0176DE, azul #173F8A, azul oscuro: #03122E, amarillo: #FEC60D, amarillo oscuro: #E3AE00 -->
<!--
<figure>
<img align ="right" src="IMClogo.png" alt="logo" width="250" height="400"><br><br><br><br><br>
</figure>
 -->

<h2 style="color:#03122E;text-align:center;"> Capitulo 3. Metodos para Leyes de conservacion <br> </h2>

<hr style="border:3px solid #E3AE00 "> </hr>

In [1]:
# use netgen.meshing module for 1d meshes
from ngsolve import *
#from netgen.meshing import Mesh as Mesh1d
#from netgen.meshing import MeshPoint,Element0D, Element1D
from netgen.csg import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib as plt
from math import pi
from netgen.geom2d import SplineGeometry

In [2]:
class problem2D_advection:
    def __init__(self, problemnumber=1):
        if problemnumber==1:
            self.problem1()
        elif problemnumber==2:
            self.problem2()
            
    def problem1(self):
        self.geo = SplineGeometry()
        self.geo.AddRectangle( (0, 0), (1, 1), bcs = ("bottom", "right", "top", "left"))
        
        self.tpar = Parameter(0.0)
        self.b = CoefficientFunction((1,0))
        self.ubnd = sin(2*pi*(-self.tpar))
        self.u0 = sin(2*pi*x)
        
    def problem2(self):
        self.geo = SplineGeometry()
        self.geo.AddRectangle( (0, 0), (1, 1), bcs = ("bottom", "right", "top", "left"))
        
        self.b = CoefficientFunction((1+sin(4*pi*y),2))
        self.ubnd = IfPos(x-0.125,IfPos(0.625-x,1+cos(8*pi*x),0),0)
        self.u0 = 0
        self.tpar = Parameter(0.0)
    
    def problem2(self):
        self.geo = SplineGeometry()
        self.geo.AddRectangle( (0, 0), (1, 1), bcs = ("bottom", "right", "top", "left"))
        
        self.b = CoefficientFunction((1+sin(4*pi*y),2))
        self.ubnd = IfPos(x-0.125,IfPos(0.625-x,1+cos(8*pi*x),0),0)
        self.u0 = 0
        self.tpar = Parameter(0.0)

        
        
#mesh = Mesh( geo.GerateMesh(maxh=0.125))
#Draw(mesh)
#order = 4

In [9]:
def DGmethod(problem, mesh, tend=0.6, order= 3, CFL=0.25):
    
    # Create bilinear form
    VT = L2(mesh,order=order)
    u,v = VT.TnT()
    a = BilinearForm(VT, nonassemble=True)
    n = specialcf.normal(mesh.dim)
    upw_flux = problem.b*n * IfPos(problem.b*n, u, u.Other(bnd=problem.ubnd))
    dS = dx(element_boundary=True)
    a += - problem.b * grad(v) * u * dx
    a += upw_flux * v * dS
    
    # initial condition
    #gfut = GridFunction(VT, multidim=0)
    gfu = GridFunction(VT)
    gfu.Set(problem.u0)
    #gfut.AddMultiDimComponent(gfu.vec)
    sceneu=Draw(gfu,mesh,"u", min=-0.5, max=2.0, animate=False, order=5)
    
    # time loop
    invm = VT.Mass(1).Inverse()
    res = gfu.vec.CreateVector()
    
    elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
    h = max([(2*vol)**(1/2) for vol in elvol]) # 2D
    t = 0; dt = CFL*h 
    
    while t < tend-0.5*dt:
        res.data = invm @ a.mat * gfu.vec
        gfu.vec.data -= dt * res
        t += dt
        problem.tpar.Set(t)
        sceneu.Redraw()
        #gfut.AddMultiDimComponent(gfu.vec)
    return gfu

In [10]:
problem1 = problem2D_advection(problemnumber=1)
mesh = Mesh( problem1.geo.GenerateMesh(maxh=0.125))
gfut = DGmethod(problem1, mesh, tend=1.0, order= 4, CFL=0.125)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [11]:
problem2 = problem2D_advection(problemnumber=2)
mesh = Mesh( problem2.geo.GenerateMesh(maxh=0.125))
gfut = DGmethod(problem2, mesh, tend=0.6, order= 4, CFL=1/500)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…